# Prepare sample

In [1]:
import os
import numpy as np
import pandas as pd
from root_numpy import root2array

Welcome to ROOTaaS 6.06/06


## Setup sample paths

In [2]:
prefix = "/Users/chris/THORr21/"

sig_train_root = prefix + "sig1P_train.root"
sig_test_root = prefix + "sig1P_test.root"
bkg_train_root = prefix + "bkg1P_weight_train.root"
bkg_test_root = prefix + "bkg1P_weight_test.root"

In [3]:
for file in [sig_train_root, sig_test_root, bkg_train_root, bkg_test_root]:
    if os.path.exists(file):
        print("[OK]: {}".format(file))
    else:
        print("[MISSING]: {}".format(file))

[OK]: /Users/chris/THORr21/sig1P_train.root
[OK]: /Users/chris/THORr21/sig1P_test.root
[OK]: /Users/chris/THORr21/bkg1P_weight_train.root
[OK]: /Users/chris/THORr21/bkg1P_weight_test.root


## Merge samples

In [4]:
for s_root, b_root, label in [(sig_train_root, bkg_train_root, "train"), (sig_test_root, bkg_test_root, "test")]:
    s = pd.DataFrame(root2array(s_root, treename="CollectionTree"))
    s["is_sig"] = 1
    b = pd.DataFrame(root2array(b_root, treename="CollectionTree"))
    b["is_sig"] = 0
    
    # Probably not necessary
    ws = s.weight.sum()
    wb = b.weight.sum()

    if ws > wb:
        s.weight *= wb / ws
    else:
        b.weight *= ws / wb

    df = pd.concat([s, b])
    df.to_hdf("samples/{}.h5".format(label), "CollectionTree", format="t", complevel=9, complib="zlib")